In [1]:
import os, sys
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "intelliwiz_config,settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [2]:
# Model imports
from apps.activity.models import Asset, Attachment, Device, DeviceEventlog, Event, Job, Jobneed, JobneedDetails, Location, Question, QuestionSet, QuestionSetBelonging, WorkPermit
from apps.attendance.models import PeopleEventlog, TestGeo, Tracking
from apps.onboarding.models import Bt, Contract, ContractDetail, GeofenceMaster, Shift, SitePeople, TypeAssist, WizardDraft
from apps.peoples.models import Capability, People, PermissionGroup, Pgbelonging, Pgroup
from apps.reminder.models import Reminder
from apps.tenants.models import Tenant
from apps.y_helpdesk.models import EscalationMatrix, Ticket

# Shell Plus Django Imports
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When, ExpressionWrapper
from django.utils import timezone
from django.urls import reverse
from django.db.models import Exists, OuterRef, Subquery, FloatField
from datetime import datetime, timedelta

In [8]:
from apps.work_order_management import utils

In [4]:
from django.urls import reverse
reverse('onboarding:typeassist')

 2023-03-24 02:40:49,055  WARNING     from method: ModelSchemaField                  << <class 'django.contrib.gis.db.models.fields.PointField'> is currently unhandled, defaulting to str. >>


'/onboarding/typeassist/'

In [33]:
# A python function that takes timezone offset as an input and returns timezone
import pytz # A library to handle timezones

def get_timezone(offset):  # sourcery skip: aware-datetime-for-utc
    import pytz
    from datetime import datetime, timedelta
    # Convert the offset string to a timedelta object
    offset = f'+{offset}' if offset > 0 else str(offset)
    sign = offset[0] # The sign of the offset (+ or -)
    mins = int(offset[1:])
    delta = timedelta(minutes=mins) # The timedelta object
    if sign == "-": # If the sign is negative, invert the delta
        delta = -deltaobj = 

    # Loop through all the timezones and find the ones that match the offset
    matching_zones = [] # A list to store the matching zones
    for zone in pytz.all_timezones: # For each timezone
        tz = pytz.timezone(zone) # Get the timezone object
        utc_offset = tz.utcoffset(datetime.utcnow()) # Get the current UTC offset
        if utc_offset == delta: # If the offset matches the input
            matching_zones.append(zone) # Add the zone to the list

    # Return the list of matching zones or None if no match found
    return matching_zones if matching_zones else None


In [17]:
Jobneed.objects.get_expired_jobs().values_list('id', flat=True)



<QuerySet []>

In [3]:
from background_tasks.tasks import autoclose_job


In [4]:
autoclose_job()

 2023-04-24 00:55:46,092  INFO        from method: autoclose_job                     << notifying through email... >>
 2023-04-24 00:55:46,098  INFO        from method: autoclose_job                     << recipents are as follows...['damola@tourtrax.ca'] >>
 2023-04-24 00:55:46,099  INFO        from method: autoclose_job                     << ticket needs to be generated >>
 2023-04-24 00:55:46,108  INFO        from method: autoclose_job                     << {'ticketcategory_id': 238, 'client_id': 31, 'level': 0, 'bu_id': 32, 'id': 287, 'cdtz': datetime.datetime(2023, 4, 24, 5, 55, 46, tzinfo=<UTC>), 'ctzoffset': -1} >>
 2023-04-24 00:55:46,137  INFO        from method: autoclose_job                     << mail sent, record_id:6436 >>
 2023-04-24 00:55:46,150  INFO        from method: autoclose_job                     << notifying through email... >>
 2023-04-24 00:55:46,156  INFO        from method: autoclose_job                     << recipents are as follows...['namrata.shahid@y

 2023-04-24 00:55:46,408  INFO        from method: autoclose_job                     << {'ticketcategory_id': 238, 'client_id': 31, 'level': 0, 'bu_id': 32, 'id': 288, 'cdtz': datetime.datetime(2023, 4, 24, 5, 55, 46, tzinfo=<UTC>), 'ctzoffset': -1} >>
 2023-04-24 00:55:46,412  INFO        from method: autoclose_job                     << mail sent, record_id:6479 >>
 2023-04-24 00:55:46,417  INFO        from method: autoclose_job                     << notifying through email... >>
 2023-04-24 00:55:46,422  INFO        from method: autoclose_job                     << recipents are as follows...['damola@tourtrax.ca'] >>
 2023-04-24 00:55:46,424  INFO        from method: autoclose_job                     << ticket needs to be generated >>
 2023-04-24 00:55:46,432  INFO        from method: autoclose_job                     << {'ticketcategory_id': 238, 'client_id': 31, 'level': 0, 'bu_id': 32, 'id': 289, 'cdtz': datetime.datetime(2023, 4, 24, 5, 55, 46, tzinfo=<UTC>), 'ctzoffset': -1} >

 2023-04-24 00:55:46,651  INFO        from method: autoclose_job                     << notifying through email... >>
 2023-04-24 00:55:46,656  INFO        from method: autoclose_job                     << recipents are as follows...['namrata.shahid@youtility.in', 'namrata.shahid@gmail.com', 'n.shahid@yahoo.co.in'] >>
 2023-04-24 00:55:46,659  INFO        from method: autoclose_job                     << mail sent, record_id:6513 >>
 2023-04-24 00:55:46,665  INFO        from method: autoclose_job                     << notifying through email... >>
 2023-04-24 00:55:46,670  INFO        from method: autoclose_job                     << recipents are as follows...['namrata.shahid@youtility.in', 'namrata.shahid@gmail.com', 'n.shahid@yahoo.co.in'] >>
 2023-04-24 00:55:46,673  INFO        from method: autoclose_job                     << mail sent, record_id:6514 >>
 2023-04-24 00:55:46,679  INFO        from method: autoclose_job                     << notifying through email... >>
 2023-04

 2023-04-24 00:55:46,932  INFO        from method: autoclose_job                     << mail sent, record_id:6491 >>
 2023-04-24 00:55:46,938  INFO        from method: autoclose_job                     << notifying through email... >>
 2023-04-24 00:55:46,942  INFO        from method: autoclose_job                     << recipents are as follows...['namrata.shahid@youtility.in', 'namrata.shahid@gmail.com', 'n.shahid@yahoo.co.in'] >>
 2023-04-24 00:55:46,945  INFO        from method: autoclose_job                     << mail sent, record_id:6492 >>
 2023-04-24 00:55:46,950  INFO        from method: autoclose_job                     << notifying through email... >>
 2023-04-24 00:55:46,954  INFO        from method: autoclose_job                     << recipents are as follows...['namrata.shahid@youtility.in', 'namrata.shahid@gmail.com', 'n.shahid@yahoo.co.in'] >>
 2023-04-24 00:55:46,956  INFO        from method: autoclose_job                     << mail sent, record_id:6493 >>
 2023-04-

{'story': "total expired jobs = 99\nusing database: default\nprocessing record with id= 6436\nrecord category is RAISETICKETNOTIFY\nemail recipents are as follows ['damola@tourtrax.ca']\ncontext in email template is {'subject': 'AUTOCLOSE TASK planned on                     23-Apr-2023 09:00 not reported in time', 'buname': 'Branch Office Nigeria', 'plan_dt': '23-Apr-2023 09:00', 'creatorname': 'Damola', 'assignedto': 'Damola [PEOPLE]', 'exp_dt': '23-Apr-2023 09:20', 'show_ticket_body': True, 'identifier': 'TASK', 'jobdesc': 'Power Generator Check', 'ticketno': 287, 'tjobdesc': 'AUTOCLOSED TASK planned on 23-Apr-2023 09:00 not reported in time', 'categoryname': 'Raise Ticket & Notify', 'priority': 'MEDIUM', 'status': 'NEW', 'tcreatedby': 'Damola', 'created_at': '24-Apr-2023 05:54', 'tkt_assignedto': 'Damola [PEOPLE]'}\nprocessing record with id= 6327\nrecord category is AUTOCLOSENOTIFY\nemail recipents are as follows ['namrata.shahid@youtility.in', 'namrata.shahid@gmail.com', 'n.shahid